In [2]:
import os
import time
import torch
import torch.nn as nn
from torch.distributed.fsdp.wrap import transformer_auto_wrap_policy
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from functools import partial
import hydra
from omegaconf import DictConfig, OmegaConf


from model.tokenizer import (CausalTokenizerDecoder, 
                             CausalTokenizerEncoder, 
                             CausalTokenizerConfig, 
                             TokensToImageHead, 
                             ImagePatchifier)
from model.blocks import EfficientTransformerLayer
from model.utils import TokenMasker

from dataset import ShardedHDF5Dataset, SingleViewSequenceDataset

class ModelWrapper(nn.Module):
    def __init__(self, cfg:DictConfig):
        super().__init__()
        self.cfg = cfg
        tokenizer_cfg = CausalTokenizerConfig(**OmegaConf.to_object(cfg.tokenizer)) 
        self.encoder = CausalTokenizerEncoder(tokenizer_cfg)
        self.decoder = CausalTokenizerDecoder(tokenizer_cfg)
        self.patchifier = ImagePatchifier(cfg.tokenizer.patch_size, cfg.tokenizer.model_dim)
        self.image_head = TokensToImageHead(cfg.tokenizer.model_dim, cfg.dataset.resolution, cfg.tokenizer.patch_size)
        self.masker = TokenMasker(cfg.tokenizer.model_dim, cfg.tokenizer.num_modality_tokens)

    def forward(self, images):
        images = (images*2.)-1. # Translate the images in +-1 range
        tokens = self.patchifier(images)
        masked_tokens = self.masker(tokens)
        z, _ = self.encoder(masked_tokens)
        z_decoded = self.decoder(z)
        recon_images = self.image_head(z_decoded)
        return  torch.clamp((recon_images + 1)/2., 0., 1.)

In [3]:
from hydra import initialize, compose
with initialize(config_path="config"):
    cfg = compose('tokenizer_large_cfg1.yaml')
model = ModelWrapper(cfg)

/tmp/ipykernel_2171087/4048375112.py:2: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="config"):
/home/mim-server/miniconda3/envs/dreamerv4/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'tokenizer_large_cfg1.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
import torch
state_dict = torch.load("/home/mim-server/projects/rooholla/dreamer-v4/tokenizer_ckpts/large/cfg2/19.pt")
# state_dict = torch.load("/home/mim-server/dreamer-v4/tokenizer_ckpts/2025-11-28_21-08-18-cfg2/checkpoints/checkpoint_step_0342993.pt")
# state_dict = torch.load("/home/mim-server/dreamer-v4/tokenizer_ckpts/pusht_tokenizer_small.pt")
sd = state_dict['model']

# Clean the keys (remove `_orig_mod.`)
clean_sd = {}
for k, v in sd.items():
    new_k = k.replace("_orig_mod.", "")
    clean_sd[new_k] = v

In [5]:
num_params = sum(p.numel() for p in model.encoder.parameters() if p.requires_grad)
print(f"Number of encoder parameters (M): {num_params/1e6:.2f}M")
num_params = sum(p.numel() for p in model.decoder.parameters() if p.requires_grad)
print(f"Number of decoder parameters (M): {num_params/1e6:.2f}M")

Number of encoder parameters (M): 201.62M
Number of decoder parameters (M): 201.62M


In [6]:
model.load_state_dict(clean_sd)
model = model.cuda()
model = model.eval()

In [7]:
# dataset = ShardedHDF5Dataset(
#     data_dir="/home/mim-server/soar_data_sharded/soar_data_sharded",
#     window_size=64,
#     stride=1,
#     split="train",
#     train_fraction=0.9,
#     split_seed=123,
# )   
from dataset import ShardedHDF5Dataset, SingleViewSequenceDataset

pusht_dataset = SingleViewSequenceDataset("/home/mim-server/datasets/pushT/224/episode_0.h5", 64, load_to_ram=True, non_overlapping=True)

In [8]:
from torch.nn.functional import interpolate

# imgs = dataset[1500]['image'].cuda()
imgs = pusht_dataset[10]['observation.image'].cuda()
imgs = interpolate(imgs, (256,256))
imgs1=imgs[None].to(torch.bfloat16)[:,:32,...]
imgs2=imgs[None].to(torch.bfloat16)[:,32:,...]
imgs3=imgs[None].to(torch.bfloat16)[:,16:48,...]
with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        patches = model.patchifier((imgs1*2.)-1.)
        latents1, _ = model.encoder(patches)
        
        patches = model.patchifier((imgs2*2.)-1.)
        latents2, _ = model.encoder(patches)

        patches = model.patchifier((imgs3*2.)-1.)
        latents3, _ = model.encoder(patches)

        # z_decoded = model.decoder(latents)
        # imgs_recon = model.image_head(z_decoded)
        # imgs_recon = (imgs_recon + 1)/2.

# import matplotlib.pyplot as plt
# vis_imgs = torch.concat([imgs, imgs_recon], dim=-2).squeeze().cpu().to(torch.float32)
# time_idx = [i*5 for i in range(6)]
# plt.imshow(torch.concat([vis_imgs[idx] for idx in time_idx], dim=-1).permute(1,2,0))
# from torchvision.utils import save_image
# save_image(torch.concat([vis_imgs[idx] for idx in time_idx], dim=-1), 'vis.png')

In [9]:
letents_concatinated = torch.cat([latents1[:, 16:], latents2[:, :16]], dim=1)
letents_computed = latents3

# _ = plt.plot((letents_concatinated-letents_computed).to(torch.float32).cpu().view(-1))

In [10]:
import mediapy
with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        z_decoded = model.decoder(letents_concatinated)
        imgs_recon = model.image_head(z_decoded)
        imgs1_recon = (imgs_recon + 1)/2.

        z_decoded = model.decoder(letents_computed)
        imgs_recon = model.image_head(z_decoded)
        imgs2_recon = (imgs_recon + 1)/2.

vis_imgs = torch.concat([imgs1_recon, imgs2_recon], dim=-1).squeeze().cpu().to(torch.float32)
mediapy.show_video([(vis_imgs[idx].permute(1,2,0)*255).to(torch.uint8).numpy() for idx in range(vis_imgs.shape[0])], fps=10)

This browser does not support the video tag.


In [31]:
from torch.nn.functional import interpolate
import mediapy

# imgs = dataset[1500]['image'].cuda()
K = 12
imgs = pusht_dataset[K]['observation.image'].cuda()
latents = pusht_dataset[K]['observation.tokens'].cuda()
imgs = interpolate(imgs, (256,256))
imgs=imgs[None].to(torch.bfloat16)
latents=latents[None].to(torch.bfloat16)
signal = 0.99
with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        z_decoded = model.decoder(computed_tokens)
        imgs_recon1 = model.image_head(z_decoded)
        imgs_recon1 = (imgs_recon1 + 1)/2.

        z_decoded = model.decoder(ds_tokens[None].cuda().to(torch.bfloat16))
        imgs_recon2 = model.image_head(z_decoded)
        imgs_recon2 = (imgs_recon2 + 1)/2.       

import matplotlib.pyplot as plt
vis_imgs = torch.concat([imgs_recon1, imgs_recon2], dim=-1).squeeze().cpu().to(torch.float32)
mediapy.show_video([(vis_imgs[idx].permute(1,2,0)*255).to(torch.uint8).numpy() for idx in range(vis_imgs.shape[0])], fps=10)

NameError: name 'computed_tokens' is not defined

In [11]:
import matplotlib.pyplot as plt
_ = plt.hist(latents.view(-1).cpu().to(torch.float32), bins=100)



NameError: name 'latents' is not defined

In [11]:
from torch.nn.functional import interpolate
import mediapy

# imgs = dataset[1500]['image'].cuda()
K = 12
imgs = pusht_dataset[K]['observation.image'].cuda()
latents = pusht_dataset[K]['observation.tokens'].cuda()
imgs = interpolate(imgs, (256,256))
imgs=imgs[None].to(torch.bfloat16)
latents=latents[None].to(torch.bfloat16)
signal = 0.99
with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        latents = signal*latents + (1-signal)*torch.randn_like(latents).cuda()
        z_decoded = model.decoder(latents)
        imgs_recon = model.image_head(z_decoded)
        imgs_recon = (imgs_recon + 1)/2.

import matplotlib.pyplot as plt
vis_imgs = torch.concat([imgs, imgs_recon], dim=-1).squeeze().cpu().to(torch.float32)
mediapy.show_video([(vis_imgs[idx].permute(1,2,0)*255).to(torch.uint8).numpy() for idx in range(vis_imgs.shape[0])], fps=10)

This browser does not support the video tag.


In [12]:
from model.sampler import sample_dreamer4_trajectory, shortcut_euler_step_last_frame
from model.dynamics import *
from omegaconf import DictConfig, OmegaConf


from hydra import initialize, compose
with initialize(config_path="config"):
    dynamics_cfg = compose('dynamics_small.yaml')

denoiser_cfg = DreamerV4DenoiserCfg(**OmegaConf.to_object(dynamics_cfg.denoiser))
denoiser = DreamerV4Denoiser(denoiser_cfg)
device = 'cuda:0'
denoiser = denoiser.to(device)
# denoiser = torch.compile(denoiser, mode="max-autotune", fullgraph=False)

/tmp/ipykernel_2171087/1641590848.py:7: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="config"):
/home/mim-server/miniconda3/envs/dreamerv4/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dynamics_small.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [24]:
import torch
state_dict = torch.load("/home/mim-server/projects/rooholla/dreamer-v4/dynamics_ckpts/2025-12-04_23-37-40/checkpoints/checkpoint_step_0210444.pt")
# state_dict = torch.load("/home/mim-server/dreamer-v4/tokenizer_ckpts/pusht_tokenizer_small.pt")
sd = state_dict['model']

# Clean the keys (remove `_orig_mod.`)
clean_sd = {}
for k, v in sd.items():
    new_k = k.replace("_orig_mod.", "")
    clean_sd[new_k] = v

denoiser.load_state_dict(clean_sd)

<All keys matched successfully>

In [25]:
@torch.no_grad()
def sample_video_snippet(
    denoiser,
    latents,
    act,
    batch_size: int,
    seq_len: int,
    device: torch.device,
    num_steps: int = 4,
    K_max: int = 64,
    latent_shape: tuple = (256, 32), # (N_lat, D_lat) - UPDATE THIS based on your config
):
    """
    Generates a video snippet (latents) from pure noise using the shortcut schedule.
    """
    B, T = batch_size, seq_len
    N_lat, D_lat = latent_shape
    
    # 1. Initialize pure noise (τ=0)
    z = torch.randn(B, T, N_lat, D_lat, device=device, dtype=torch.bfloat16)
    z[:, :latents.shape[1], ...] = latents
    # 2. Setup dyadic grid parameters
    # d = 1 / num_steps
    step_size = 1.0 / float(num_steps)
    
    max_pow2 = int(math.log2(K_max))
    step_index_raw = int(math.log2(num_steps)) 
    
    # The integer index representing step size 'd' passed to the model
    # For K_max=32 (pow=5) and steps=4 (pow=2), index is 5-2=3.
    current_step_index = max_pow2 - step_index_raw
    
    step_index_tensor = torch.full((B, T), current_step_index, dtype=torch.long, device=device)
    d_min = 1.0 / K_max

    print(f"🎨 Sampling: Batch={B}, T={T}, Steps={num_steps} (d={step_size:.4f})")
    act = act.to(torch.bfloat16).cuda()
    # 3. Sampling Loop
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        for k in range(num_steps):
            tau_val = k * step_size
            
            # Compute integer index for tau on the finest grid
            current_tau_index_val = int(round(tau_val / d_min))
            tau_index_tensor = torch.full((B, T), current_tau_index_val, dtype=torch.long, device=device)
            
            print(f"   Step {k+1}/{num_steps}: tau={tau_val:.2f} -> index={current_tau_index_val}")
            # Predict clean z_1

            z_hat = apply_denoiser(
                denoiser,
                x_tau=z,
                tau_index=tau_index_tensor,
                step_index=step_index_tensor,
                no_grad=True,
                continuous_actions=act,
            )
            
            # 4. Update z_tau to z_{tau+d}
            if k == num_steps - 1:
                z = z_hat
            else:
                # v = (z_1 - z_tau) / (1 - tau)
                denom = 1.0 - tau_val
                velocity = (z_hat - z) / denom
                z = z + velocity * step_size
            
    return z.to(torch.float32)
# denoiser(latents, )

In [ ]:
import time
act = torch.zeros(1, 32, 2)
act[0, :16, 1] = 0.1
tic = time.time()
sampled_latent = sample_video_snippet(denoiser, latents2[:,10].unsqueeze(1).repeat(1, 31, 1, 1), act, 1, 32, 'cuda:0', 4, 128, (256, 32))
print(time.time()-tic)

🎨 Sampling: Batch=1, T=32, Steps=4 (d=0.2500)
   Step 1/4: tau=0.00 -> index=0
   Step 2/4: tau=0.25 -> index=32
   Step 3/4: tau=0.50 -> index=64
   Step 4/4: tau=0.75 -> index=96
0.07694125175476074


In [31]:

with torch.no_grad():
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        z_decoded = model.decoder(sampled_latent)
        imgs_recon = model.image_head(z_decoded)
        imgs_recon = (imgs_recon + 1)/2.

import matplotlib.pyplot as plt
imgs_recon = imgs_recon.cpu().to(torch.float32)
vid = [(imgs_recon[0][idx].permute(1,2,0)*255).to(torch.uint8).numpy() for idx in range(imgs_recon[0].shape[0])]
mediapy.show_video(vid, fps=10)

This browser does not support the video tag.
